In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 20 12:27:53 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070       WDDM | 00000000:2B:00.0  On |                  N/A |
|  0%   52C    P0               56W / 270W|   1067MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 34.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# %%capture
# !pip install keras_applications
# !pip install keras_vggface
# filename = "/usr/local/lib/python3.10/dist-packages/keras_vggface/models.py"
# text = open(filename).read()
# open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
# !pip install mtcnn

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import PIL
from keras.utils import load_img, img_to_array
import os
from sklearn.metrics import accuracy_score
import cv2

In [2]:
data = pd.read_csv('../data/data.csv')
data.head()

,Unnamed: 0,bmi,gender,is_training,name
0,0,34.207396,Male,1,img_0.bmp
1,1,26.453720,Male,1,img_1.bmp
2,2,34.967561,Female,1,img_2.bmp
3,3,22.044766,Female,1,img_3.bmp
4,4,37.758789,Female,1,img_4.bmp


## MTCNN

In [3]:
def image_resize(image, size):
    # Get the dimensions of the image
    height, width, _ = image.shape
    # Calculate the aspect ratio
    aspect_ratio = width / height
    # Determine the resizing dimensions while maintaining the aspect ratio
    if aspect_ratio > 1:
        new_width = size
        new_height = int(new_width / aspect_ratio)
    elif aspect_ratio == 1:
        new_width = size
        new_height = size
    else:
        new_height = size
        new_width = int(new_height * aspect_ratio)
    # Resize the image using the determined dimensions
    resized_image = cv2.resize(image, (new_width, new_height))

    # Create a black canvas of the desired size
    padded_image = np.zeros((size, size, 3), dtype=np.uint8)
    # Calculate the padding values
    pad_top = (size - new_height) // 2
    pad_bottom = size - new_height - pad_top
    pad_left = (size - new_width) // 2
    pad_right = size - new_width - pad_left
    # Copy the resized image onto the canvas with padding
    padded_image[pad_top:pad_top+new_height, pad_left:pad_left+new_width] = resized_image
    return padded_image

In [4]:
import os
import cv2
from mtcnn import MTCNN
from keras.layers.pooling import GlobalAveragePooling2D
import pandas as pd
from keras.utils.layer_utils import get_source_inputs
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


# Load the CSV data


# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Set up the training data generator
directory = '../../Project/data/images/'

# Create a new folder to save the cropped face images
cropped_directory = '../data/CroppedImages/'
os.makedirs(cropped_directory, exist_ok=True)

# Create an instance of MTCNN
mtcnn = MTCNN()

# Function to crop faces from images and save them to a new folder
def save_cropped_faces(image_path, save_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            raise Exception
            return
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
            x = x - 20 if x -20 > 0 else x
            y = y-20 if y -20 > 0 else y
            w = w + 20
            h = h + 20 
            # Calculate the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]
            face = image_resize(face, size=100)
            # Resize the face region to fit the desired padding size
            # resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            cv2.imwrite(save_path, face)
    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))


# Iterate through the data and crop faces from images
for index, row in data.iterrows():
    image_path = os.path.join(directory, row['name'])
    save_path = os.path.join(cropped_directory, row['name'])
    save_cropped_faces(image_path, save_path)

1/1 [==============================] - 0s 15ms/step
Invalid image: ../../Project/data/images/img_4.bmp
Error processing image: ../../Project/data/images/img_4.bmp

Invalid image: ../../Project/data/images/img_5.bmp
Error processing image: ../../Project/data/images/img_5.bmp

1/1 [==============================] - 0s 19ms/step
Invalid image: ../../Project/data/images/img_40.bmp
Error processing image: ../../Project/data/images/img_40.bmp

1/1 [==============================] - 0s 20ms/step
Invalid image: ../../Project/data/images/img_86.bmp
Error processing image: ../../Project/data/images/img_86.bmp

1/1 [==============================] - 0s 14ms/step
Invalid image: ../../Project/data/images/img_93.bmp
Error processing image: ../../Project/data/images/img_93.bmp

1/1 [==============================] - 0s 14ms/step
Invalid image: ../../Project/data/images/img_112.bmp
Error processing image: ../../Project/data/images/img_112.bmp

Invalid image: ../../Project/data/images/img_113.bmp
Error

## Train for predicting age

AGE

In [6]:
import re
df = {
    "filename":[],
    "Age":[]
}
for filename in os.listdir("../data/UTKFace/"):
    label = filename.split("_")
    df["filename"].append(filename)
    df["Age"].append(label[0])

In [11]:
dfs = pd.DataFrame(df)

In [19]:
dfs['Age'] = dfs['Age'].astype(np.float32)

In [20]:
dfs

,filename,Age
0,100_0_0_20170112213500903.jpg.chip.jpg,100.0
1,100_0_0_20170112215240346.jpg.chip.jpg,100.0
2,100_1_0_20170110183726390.jpg.chip.jpg,100.0
3,100_1_0_20170112213001988.jpg.chip.jpg,100.0
4,100_1_0_20170112213303693.jpg.chip.jpg,100.0
...,...,...
23703,9_1_3_20161220222856346.jpg.chip.jpg,9.0
23704,9_1_3_20170104222949455.jpg.chip.jpg,9.0
23705,9_1_4_20170103200637399.jpg.chip.jpg,9.0
23706,9_1_4_20170103200814791.jpg.chip.jpg,9.0


In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras_vggface.vggface import VGGFace
import pandas as pd

In [13]:
def custom_loss(y_true, y_pred):
        penalty_overestimation_10 = 10.0  # Penalty factor for overestimation
        penalty_underestimation_10 = 10.0  # Penalty factor for underestimation by 10
        penalty_overestimation_5 = 5.0  # Penalty factor for overestimation
        penalty_underestimation_5 = 5.0  # Penalty factor for underestimation by 5
        
        squared_difference = tf.square(y_true - y_pred)
        overestimation_penalty_10 = tf.maximum(0.0, y_pred - y_true) * penalty_overestimation_10
        underestimation_penalty_10 = tf.maximum(0.0, y_true - y_pred - 10) * penalty_underestimation_10
        underestimation_penalty_5 = tf.maximum(0.0, y_true - y_pred - 5) * penalty_underestimation_5
        overestimation_penalty_5 = tf.maximum(0.0, y_pred - y_true) * penalty_overestimation_5
        total_loss = tf.reduce_mean(squared_difference + overestimation_penalty_10 + overestimation_penalty_5 + underestimation_penalty_10 + underestimation_penalty_5, axis=-1)
        
        return total_loss

# Register the custom loss function
keras.utils.get_custom_objects()['custom_loss'] = custom_loss

def create_bmi_model(freeze = 2):
    base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[:-1*freeze]:
        layer.trainable = False
    # Add custom layers on top of the base model
    x = base_model.layers[-1].output
    x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
    x = keras.layers.AveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(units=128, activation='relu')(x)
    predictions = Dense(1, activation='relu')(x)  # Use linear activation for BMI prediction
    model = Model(inputs=base_model.input, outputs=predictions)
    

    # Compile the model with the custom loss function
    model.compile(optimizer='adam', loss=custom_loss, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
cropped_directory = "../data/UTKFace/"
train_set = train_datagen.flow_from_dataframe(
        dataframe=dfs,
        directory=cropped_directory,
        x_col='filename',
        y_col='Age',
        target_size=(224, 224),
        batch_size=64,
        class_mode='other',
        subset='training')

validation_set = train_datagen.flow_from_dataframe(
        dataframe=dfs,
        directory=cropped_directory,
        x_col='filename',
        y_col='Age',
        target_size=(224, 224),
        batch_size=64,
        class_mode='other',
        subset='validation')

for i in range(2,4,1):
      print(i)
      bmi_model = create_bmi_model(i)
      checkpoint_bmi = ModelCheckpoint('best_age_model_v1_'+str(i)+'.h5', monitor='val_rmse', save_best_only=True, verbose=1)
      print(bmi_model.summary())
      history = bmi_model.fit(
        train_set,
        epochs=50,
        callbacks=[checkpoint_bmi],
        validation_data=validation_set
    )

Found 18967 validated image filenames.
Found 4741 validated image filenames.
2
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                              